#Korean-Text-to-SQL

### install requirements

In [2]:
##!git clone https://github.com/naver/sqlova.git
%cd drive/MyDrive/sqlovaa/sqlova

/content/drive/MyDrive/sqlovaa/sqlova


In [3]:
# SQLova requirements

!pip3 install sqlalchemy==1.3
!pip3 install records==0.5.2
!pip3 install babel matplotlib defusedxml tqdm
!pip3 install ujson stanza

import stanza
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
#stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

#!ls $CORENLP_HOME

!pip3 install konlpy

     |████████████████████████████████| 5.9 MB 5.1 MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.0-cp37-cp37m-linux_x86_64.whl size=1182825 sha256=142a4c2072f61e3c6a26b9032778cd081b3ba6b51bcabf31fdcb50a9a1e7cfb5
  Stored in directory: /root/.cache/pip/wheels/cb/b4/c8/1ae5d5a576b48cf3ea64943280034378cd76055c0f207bc4c1
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.25
    Uninstalling SQLAlchemy-1.4.25:
      Successfully uninstalled SQLAlchemy-1.4.25
     |████████████████████████████████| 47 kB 2.5 MB/s 
     |████████████████████████████████| 214 kB 5.0 MB/s 
     |████████████████████████████████| 432 kB 57.2 MB/s 
     |████████████████████████████████| 170 kB 72.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=7ed78d9a1c0120fb66193821ab795838e32060fe25c713b523ebc97541b4ac4e
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621e

In [4]:
# KoBERT requirements

!pip install mxnet>=1.4.0
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=67ea5476cc37553ea6f2dce7238aee15e0d0738c68bafbde16ce950c8a993c43
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
     |████████████████████████████████| 754 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 47.0 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-66_n3cfo
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-66_n3cfo
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=f932324349666f79a5f31eaaabe3dfc0b18838e4676d804e09d114ee2b01ab8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-

In [5]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.9.0+cu111
cuda version: 11.1
cudnn version:8005


### 1st Level Tokenization

In [ ]:
#!python3 annotate_ws.py --din data_and_model/for_test_144 --dout data_and_model/for_test_144/okt --split 'dev' #1st tokenization with CoreNLP / okt

In [ ]:
#!python3 annotate_ws.py --din data_and_model/for_test_144 --dout data_and_model/for_test_144/okt --split 'train' #1st tokenization with CoreNLP / okt

In [ ]:
#!python3 annotate_ws.py --din data_and_model/for_test_144 --dout data_and_model/for_test_144/okt --split 'test' #1st tokenization with CoreNLP / okt

###Train, Predict & Evaluate

####CoreNLP, mcs

In [26]:
!python3 train_ale.py --do_train --tepoch 40 --seed 1 --bS 4 \
    --accumulate_gradients 2 --bert_type_abb mcS --fine_tune \
    --lr 0.0001 --lr_bert 0.00001 --max_seq_leng 222

BERT-type: multi_cased_L-12_H-768_A-12
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Batch_size = 8
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: True
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Load pre-trained parameters.
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq

In [27]:
# dev
!python3 predict.py --bert_type_abb mcS \
    --model_file ../result_files/CoreNLP/mcS/model_best.pt \
    --bert_model_file ../result_files/CoreNLP/mcS/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/CoreNLP/mcS \
    --data_path ./data_and_model/for_test_144/CoreNLP \
    --split dev

!python3 evaluate_ws.py

BERT-type: multi_cased_L-12_H-768_A-12
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
./data_and_model/for_test_144/CoreNLP/dev.db
100% 15/15 [00:00<00:00, 466.38it/s]
{
  "ex_accuracy": 0.4,
  "lf_accuracy": 0.2
}


In [28]:
# test
!python3 predict.py --bert_type_abb mcS \
    --model_file ../result_files/CoreNLP/mcS/model_best.pt \
    --bert_model_file ../result_files/CoreNLP/mcS/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/CoreNLP/mcS \
    --data_path ./data_and_model/for_test_144/CoreNLP \
    --split test

!python3 evaluate_ws.py

BERT-type: multi_cased_L-12_H-768_A-12
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
./data_and_model/for_test_144/CoreNLP/test.db
100% 26/26 [00:00<00:00, 321.81it/s]
{
  "ex_accuracy": 0.38461538461538464,
  "lf_accuracy": 0.11538461538461539
}


####CoreNLP, KoBERT

In [29]:
!python3 train_ale_kobert.py --do_train --tepoch 40 --seed 1 --bS 4 \
    --accumulate_gradients 2 --bert_type_abb ko --fine_tune \
    --lr 0.0005 --lr_bert 0.00005 --max_seq_leng 222

BERT-type: kobert
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Batch_size = 8
BERT parameters:
learning rate: 5e-05
Fine-tune BERT: True
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 

In [32]:
# dev
!python3 predict_ale_kobert.py --bert_type_abb ko \
    --model_file ../result_files/CoreNLP/KoBERT/model_best.pt \
    --bert_model_file ../result_files/CoreNLP/KoBERT/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/CoreNLP/KoBERT \
    --data_path ./data_and_model/for_test_144/CoreNLP \
    --split dev

!python3 evaluate_ws.py

BERT-type: kobert
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
./data_and_model/for_test_144/CoreNLP/dev.db
100% 15/15 [00:00<00:00, 468.01it/s]
{
  "ex_accuracy": 0.4,
  "lf_accuracy": 0.13333333333333333
}


In [33]:
# test
!python3 predict_ale_kobert.py --bert_type_abb ko \
    --model_file ../result_files/CoreNLP/KoBERT/model_best.pt \
    --bert_model_file ../result_files/CoreNLP/KoBERT/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/CoreNLP/KoBERT \
    --data_path ./data_and_model/for_test_144/CoreNLP \
    --split test

!python3 evaluate_ws.py

BERT-type: kobert
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
./data_and_model/for_test_144/CoreNLP/test.db
100% 26/26 [00:00<00:00, 307.17it/s]
{
  "ex_accuracy": 0.3076923076923077,
  "lf_accuracy": 0.11538461538461539
}


####okt, mcs

In [34]:
!python3 train_ale.py --do_train --tepoch 40 --seed 1 --bS 4 \
    --accumulate_gradients 2 --bert_type_abb mcS --fine_tune \
    --lr 0.0001 --lr_bert 0.00001 --max_seq_leng 222

BERT-type: multi_cased_L-12_H-768_A-12
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Batch_size = 8
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: True
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Load pre-trained parameters.
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq

In [35]:
# dev
!python3 predict.py --bert_type_abb mcS \
    --model_file ../result_files/okt/mcS/model_best.pt \
    --bert_model_file ../result_files/okt/mcS/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/okt/mcS \
    --data_path ./data_and_model/for_test_144/okt \
    --split dev

!python3 evaluate_ws.py

BERT-type: multi_cased_L-12_H-768_A-12
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
./data_and_model/for_test_144/okt/dev.db
100% 15/15 [00:00<00:00, 439.15it/s]
{
  "ex_accuracy": 0.6,
  "lf_accuracy": 0.3333333333333333
}


In [36]:
# test
!python3 predict.py --bert_type_abb mcS \
    --model_file ../result_files/okt/mcS/model_best.pt \
    --bert_model_file ../result_files/okt/mcS/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/okt/mcS \
    --data_path ./data_and_model/for_test_144/okt \
    --split test

!python3 evaluate_ws.py

BERT-type: multi_cased_L-12_H-768_A-12
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 119547
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
./data_and_model/for_test_144/okt/test.db
100% 26/26 [00:00<00:00, 307.37it/s]
{
  "ex_accuracy": 0.5769230769230769,
  "lf_accuracy": 0.38461538461538464
}


####okt, KoBERT

In [37]:
#okt, KoBERT
!python3 train_ale_kobert.py --do_train --tepoch 40 --seed 1 --bS 4 \
    --accumulate_gradients 2 --bert_type_abb ko --fine_tune \
    --lr 0.0005 --lr_bert 0.00005 --max_seq_leng 222

BERT-type: kobert
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Batch_size = 8
BERT parameters:
learning rate: 5e-05
Fine-tune BERT: True
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 

In [38]:
# dev
!python3 predict_ale_kobert.py --bert_type_abb ko \
    --model_file ../result_files/okt/KoBERT/model_best.pt \
    --bert_model_file ../result_files/okt/KoBERT/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/okt/KoBERT \
    --data_path ./data_and_model/for_test_144/okt \
    --split dev

!python3 evaluate_ws.py

BERT-type: kobert
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
./data_and_model/for_test_144/okt/dev.db
100% 15/15 [00:00<00:00, 445.65it/s]
{
  "ex_accuracy": 0.4666666666666667,
  "lf_accuracy": 0.2
}


In [39]:
# test
!python3 predict_ale_kobert.py --bert_type_abb ko \
    --model_file ../result_files/okt/KoBERT/model_best.pt \
    --bert_model_file ../result_files/okt/KoBERT/model_bert_best.pt \
    --bert_path ./data_and_model \
    --result_path ../result_files/okt/KoBERT \
    --data_path ./data_and_model/for_test_144/okt \
    --split test

!python3 evaluate_ws.py

BERT-type: kobert
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
./data_and_model/for_test_144/okt/test.db
100% 26/26 [00:00<00:00, 308.79it/s]
{
  "ex_accuracy": 0.5,
  "lf_accuracy": 0.38461538461538464
}
